In [ ]:
#Util functions to parse results 
import os
def get_accuracy_from_file(path, prefices):
    result = [0 for _ in prefices]
    with open(path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            for i, prefix in enumerate(prefices):
                if line.startswith(prefix):
                    #Remove prefix from line 
                    line_ = line[len(prefix):]
                    result[i] = float(line_.split(" ")[1])
    return result

def get_accuracy_and_fn_sorted(directory, prefices):
    result = []
    for file in os.listdir(directory):
        if file.endswith(".txt"):
            accuracy = get_accuracy_from_file(os.path.join(directory, file), prefices)
            avg_accuracy = sum(accuracy) / len(accuracy)
            result.append((avg_accuracy, file))
    result.sort(key=lambda x: x[0], reverse=True)
    return result

def get_best_accuracy_and_fn(directory, prefices):
    result = get_accuracy_and_fn_sorted(directory, prefices)
    return result[0]

In [ ]:
!launch_scripts/baseline.sh "photo,cartoon,sketch"

"""Baseline results:
Evaluate: photo: Accuracy: 95.93 - Loss: 0.0010716533411048842
Evaluate: cartoon: Accuracy: 54.65 - Loss: 0.010657542146767772
Evaluate: sketch: Accuracy: 40.49 - Loss: 0.015146310804575324
"""

In [ ]:
#Activation shaping experiments
import os

def run_experiment(category, configurations):
  for configuration in configurations:
    alpha = configuration["a"]
    layers = configuration["LAYERS"]
    if ("EXTENSION" in configuration):
      extension = configuration["EXTENSION"]
    else:
      extension = ""
    !launch_scripts/activation_shaping_experiments.sh {category} "{extension}" {alpha} {layers}

#Experiments with single layers - try to find best alpha
def explore_different_alpha():
  c = []
  for layer in [0, 1, 4]:
    c = c + [
      {"a": 0.5, "LAYERS": f"{layer}"},
      {"a": 0.8, "LAYERS": f"{layer}"},
      {"a": 0.2, "LAYERS": f"{layer}"}
    ]
  run_experiment("photo,cartoon,sketch", c)


#Experiments with single layers
def explore_single_layers(alpha):
  c = []
  for layer in [ 6, 8, 10, 11, 14, 15]:
    c = c + [
      {"a": alpha, "LAYERS": f"{layer}"},
    ]
  run_experiment("photo,cartoon,sketch", c)

def finetune_alpha_by_layer(alphas, layer):
  c = []
  for a in alphas:
    c = c + [
      {"a": a, "LAYERS": f"{layer}"},
    ]
  run_experiment("photo,cartoon,sketch", c)

def finetune_alpha_by_layer(alphas, layer):
  c = []
  for a in alphas:
    c = c + [
      {"a": a, "LAYERS": f"{layer}"},
    ]
  run_experiment("photo,cartoon,sketch", c)

#explore_different_alpha()
#explore_single_layers(0.8)
#finetune_alpha_by_layer([0.7,0.9,0.99], 1)


In [ ]:
#Results for activation shaping exp
def print_(l):
  for i in l:
    print(i)
    
#Get results 
def get_results():
  print("Best result found")
  prefices = [
    f"TEST WITH BINARIZATIONEvaluate: {domain}: Accuracy:" for domain in ["photo", "cartoon", "sketch"]
  ]
  print("With binarization during evaluation")
  print_(get_accuracy_and_fn_sorted("record/activation_shaping_experiments/photo,cartoon,sketch", prefices))

  prefices = [
    f"TEST WITHOUT BINARIZATION Evaluate: {domain}: Accuracy:" for domain in ["photo", "cartoon", "sketch"]
  ]
  print("Without binarization during evaluation")
  print_(get_accuracy_and_fn_sorted("record/activation_shaping_experiments/photo,cartoon,sketch", prefices))

get_results()

"""Comments on the results:
>Best results are found with the activation shaping applied to the first layers, especially to the second one.
  =>Not sure but i think first layers are bigger
>Best results are found with Alpha = 0.9. Meaning that losing too much information is not good.
  In a way I think this is working kind of like a dropout layer. 
  With Alpha > 0.9 performances go down.
>"""

In [ ]:
#DOMAIN ADAPTATION STANDARD
import os
#TODO try to run with and without binarization in test mode
def run_experiment(category, configurations):
  for configuration in configurations:
    K = configuration["K"]
    record_mode = configuration["RECORD_MODE"]
    layers = configuration["LAYERS"]
    if ("EXTENSION" in configuration):
      extension = configuration["EXTENSION"]
    else:
      extension = ""
    !launch_scripts/domain_adaptation.sh {category} "{extension}" {K} {record_mode} {layers}



#Experiments with single layers - try to find most promising ones
def explore_different_layers():
  c = []
  for layer in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]:
    c = c + [
      {"K": 0, "LAYERS": f"{layer}", "RECORD_MODE": "threshold"}
    ]
  run_experiment("sketch", c)

#Experiments with single layers - try to find most promising ones
def explore_different_layers_with_topK():
  c = [
    #Simulate case of previous experiment where alpha was 0.9
    {"K": 0.9, "LAYERS": "1", "RECORD_MODE": "topk"},
  ]
  #Try other promising layers with topK
  for k in [0.4, 0.5, 0.8]:
    c = c + [
      {"K": k, "LAYERS": "1", "RECORD_MODE": "topk"},
      {"K": k, "LAYERS": "3", "RECORD_MODE": "topk"},
      {"K": k, "LAYERS": "9", "RECORD_MODE": "topk"},
      {"K": k, "LAYERS": "4", "RECORD_MODE": "topk"},
      {"K": k, "LAYERS": "12", "RECORD_MODE": "topk"},
      {"K": k, "LAYERS": "14", "RECORD_MODE": "topk"},
      {"K": k, "LAYERS": "11", "RECORD_MODE": "topk"},
      {"K": k, "LAYERS": "10", "RECORD_MODE": "topk"},
    ]
  c = [
     {"K": 0.8, "LAYERS": "15", "RECORD_MODE": "topk"},
  ]
  run_experiment("sketch", c)

def finetune_k():
  c = [
    {"K": 0.9, "LAYERS": "7", "RECORD_MODE": "topk"},
    {"K": 0.9, "LAYERS": "9", "RECORD_MODE": "topk"},
    {"K": 0.9, "LAYERS": "3", "RECORD_MODE": "topk"},
    {"K": 0.9, "LAYERS": "10", "RECORD_MODE": "topk"},

    {"K": 0.9, "LAYERS": "1", "RECORD_MODE": "topk"},
    {"K": 0.9, "LAYERS": "4", "RECORD_MODE": "topk"},

    {"K": 0.9, "LAYERS": "1,3", "RECORD_MODE": "topk"},
    {"K": 0.9, "LAYERS": "1,7", "RECORD_MODE": "topk"},
    {"K": 0.9, "LAYERS": "3,10", "RECORD_MODE": "topk"},
    {"K": 0.9, "LAYERS": "7, 9", "RECORD_MODE": "topk"},
    {"K": 0.9, "LAYERS": "7, 10", "RECORD_MODE": "topk"},
    {"K": 0.9, "LAYERS": "7,9,10", "RECORD_MODE": "topk"},
  ]
  run_experiment("sketch", c)

def experiment_progressive_application(): #L7, L3, L10;  - L1 L14
  c = [
    {"K": 0.9, "LAYERS": "3,7", "RECORD_MODE": "topk", "EXTENSION": "--apply_progressively=1"},
    {"K": 0.9, "LAYERS": "7, 10", "RECORD_MODE": "topk", "EXTENSION": "--apply_progressively=1"},
    {"K": 0.9, "LAYERS": "3,7,10", "RECORD_MODE": "topk", "EXTENSION": "--apply_progressively=1"},
    {"K": 0.9, "LAYERS": "1,7", "RECORD_MODE": "topk", "EXTENSION": "--apply_progressively=1"},
    {"K": 0.9, "LAYERS": "7,14", "RECORD_MODE": "topk", "EXTENSION": "--apply_progressively=1"},
    {"K": 0.9, "LAYERS": "1,7,14", "RECORD_MODE": "topk", "EXTENSION": "--apply_progressively=1"},
    {"K": 0.9, "LAYERS": "1,3,7", "RECORD_MODE": "topk", "EXTENSION": "--apply_progressively=1"},
    {"K": 0.9, "LAYERS": "1,3,7,10", "RECORD_MODE": "topk", "EXTENSION": "--apply_progressively=1"},
    {"K": 0.9, "LAYERS": "7,15", "RECORD_MODE": "topk", "EXTENSION": "--apply_progressively=1"},
    {"K": 0.9, "LAYERS": "1,7,15", "RECORD_MODE": "topk", "EXTENSION": "--apply_progressively=1"},
    {"K": 0.9, "LAYERS": "3,7,8", "RECORD_MODE": "topk", "EXTENSION": "--apply_progressively=1"},
    {"K": 0.9, "LAYERS": "3,7,9", "RECORD_MODE": "topk", "EXTENSION": "--apply_progressively=1"},
    {"K": 0.9, "LAYERS": "7,9", "RECORD_MODE": "topk", "EXTENSION": "--apply_progressively=1"},
  ]
  run_experiment("sketch", c)
#explore_different_layers()
#explore_different_layers_with_topK()
#finetune_k()
#experiment_progressive_application()
  
def run_for_other_domains():
  c = [ 
    {"K": 0.8, "LAYERS": "3,7", "RECORD_MODE": "topk", "EXTENSION": "--apply_progressively=1"},
    {"K": 0.9, "LAYERS": "7", "RECORD_MODE": "topk"}
  ]
  run_experiment("photo", c)
  run_experiment("cartoon", c)

run_for_other_domains()

In [ ]:
#Results for domain adaptation
def print_(l):
  for i in l:
    print(i)
    
#Get results 
def get_results():
  print("Best result found")
  prefices = [
    "TEST SIMPLE Evaluate: sketch: Accuracy:"
  ]
  print("Without binarization during evaluation")
  print_(get_accuracy_and_fn_sorted("record/domain_adaptation/sketch", prefices))

  prefices = [
    "TEST SIMPLE BINARIZED Evaluate: sketch: Accuracy:"
  ]
  print("With binarization during evaluation")
  print_(get_accuracy_and_fn_sorted("record/domain_adaptation/sketch", prefices))

get_results()


In [ ]:
#Domain adaptation: produce some statistics
#DOMAIN ADAPTATION STANDARD
import os
def run_experiment(category, configurations):
  for configuration in configurations:
    K = configuration["K"]
    record_mode = configuration["RECORD_MODE"]
    layers = configuration["LAYERS"]
    if ("EXTENSION" in configuration):
      extension = configuration["EXTENSION"]
    else:
      extension = ""
    !launch_scripts/domain_adaptation.sh {category} "{extension}" {K} {record_mode} {layers}

ext = "--print_stats=1 --layers_only_for_stats=0,1,2,3,4,5 --extra_str=FOR_STATS"
c = [
  {"K": 0.9, "LAYERS": "1,4,7,10,14,15", "RECORD_MODE": "topk", "EXTENSION": ext},
]
run_experiment("sketch", c)


In [ ]:
#Domain generalization
import os

def run_experiment(category, configurations):
  for configuration in configurations:
    K = configuration["K"]
    record_mode = configuration["RECORD_MODE"]
    layers = configuration["LAYERS"]
    if ("EXTENSION" in configuration):
      extension = configuration["EXTENSION"]
    else:
      extension = ""
    !launch_scripts/domain_generalization.sh {category} "{extension}" {K} {record_mode} {layers}

#Experiments with single layers - try to find most promising ones
def explore_different_layers():
  c = []
  for layer in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]:
    c = c + [
      {"K": 0, "LAYERS": f"{layer}", "RECORD_MODE": "threshold"}
    ]
  run_experiment("sketch", c)
#Try with randomized pairs instead of fixed ones
def explore_with_randomized_pairs():
  pass
explore_different_layers()


In [ ]:
#Domain generalization - explore results


In [ ]:
#Domain generalization: statistics and ablation tests
